In [1]:
import json
import pandas as pd
import os
import sqlite3
import re
import collections
from tqdm import tqdm
import copy
from nltk.util import ngrams

In [113]:
import nltk.corpus
import string
STOPWORDS = set(nltk.corpus.stopwords.words('english'))
STOPWORDS |= set(a for a in string.punctuation)

In [118]:
tokenizer.tokenize('"aaa')

['"', 'aaa']

In [69]:
def get_unique_column_values(conn, table, column, col_type):
    column = column.replace(' ', '_')
    try:
        if col_type == 'number':
            max_val = conn.execute('SELECT MAX(%s) from %s where TYPEOF(%s)==\'integer\' OR TYPEOF(%s)==\'real\' ;'%(column, table, column, column)).fetchall()
            min_val = conn.execute('SELECT MIN(%s) from %s ;'%(column, table)).fetchall()
            vals = [min_val[0][0], max_val[0][0]]
        elif col_type == 'time':
            max_val = conn.execute('SELECT MAX(%s) from %s ;'%(column, table)).fetchall()
            min_val = conn.execute('SELECT MIN(%s) from %s ;'%(column, table)).fetchall()
            vals = [min_val[0][0], max_val[0][0]]
        else:
            vals = conn.execute('SELECT DISTINCT(%s) from %s ;'%(column, table)).fetchall()
            vals = [val[0] for val in vals if val[0] is not None]
    except (sqlite3.Warning, sqlite3.Error, sqlite3.DatabaseError,
                sqlite3.IntegrityError, sqlite3.ProgrammingError,
                sqlite3.OperationalError, sqlite3.NotSupportedError) as e:
        print(e, column, table)
        if col_type == 'number':
            vals = [1, -1]
        else:
            vals = []
    return vals

In [85]:
def prepare_col_values(vals, tab='', col=''):
    nxt_mask = {}
    n_grams = set()
    cell_values = set()
    raw_cell_values = set()
    sub_words = collections.Counter()
    for val in tqdm(vals, desc='{} {}'.format(tab, col)):
        if val.startswith('http://') or val.startswith('https://'):
            val = 'http'
        tokens = tokenizer.tokenize(val)
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        sub_words.update(token_ids)
        normalized_token = ''
        i = 0
        words = []
        while i < len(tokens):
            if tokens[i].startswith('##'):
                normalized_token += tokens[i][2:]
            else:
                normalized_token += tokens[i]
            if normalized_token and (i+1 == len(tokens) or not tokens[i+1].startswith('##')):
                words.append(normalized_token)
                normalized_token = ''
            i += 1
        n_grams |= set(words)
        cell_values.add(' '.join(words[:5]))
#         for n in range(2,4):
#             n_grams |= set([' '.join(z) for z in ngrams(words, n)])
        last_token = tokenizer.bos_token_id
        for token_id in token_ids:
            if last_token not in nxt_mask:
                nxt_mask[last_token] = set()
            nxt_mask[last_token].add(token_id)
            last_token = token_id
        if last_token not in nxt_mask:
            nxt_mask[last_token] = set()
        nxt_mask[last_token].add(tokenizer.eos_token_id)
    return nxt_mask, n_grams, cell_values, sub_words

In [6]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking')
tokenizer.add_special_tokens({'bos_token':'[unused0]', 'eos_token':'[unused1]', 'unk_token':'[unused2]'})

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


0

In [ ]:
tokenizer.convert_tokens_to_ids('[unused10]')

In [ ]:
tokenizer.decode([1045, 1005, 1049, 1005, 7501, 8418, 2532, 13871, 2290, 999])

In [71]:
with open('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/tables.json','r') as f:
    db_tables = json.load(f)

In [ ]:
db_tables[0]

In [ ]:
[[x['db_id'],[[i,x['column_names'][i]] for i in range(len(x['column_types'])) if x['column_types'][i]=='time']] for x in db_tables]

In [86]:
db_values = {}
global_nxt_mask = {
    tokenizer.bos_token_id: set([
        tokenizer.convert_tokens_to_ids('true'),
        tokenizer.convert_tokens_to_ids('false'),
        tokenizer.convert_tokens_to_ids('0'),
        tokenizer.convert_tokens_to_ids('1'),
        tokenizer.convert_tokens_to_ids('2'),
        tokenizer.convert_tokens_to_ids('3'),
    ]),
    tokenizer.convert_tokens_to_ids('true'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('false'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('0'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('1'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('2'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('3'): set([tokenizer.eos_token_id]),
}
for db in db_tables:
    db_id = db['db_id']
    if db_id in ['academic', 'scholar', 'imdb', 'yelp']:
        continue
    print(db_id)
    db_col_values = [[[], [], [], [], []]]
    db_nxt_mask = copy.deepcopy(global_nxt_mask)
    db_col_value_maps = [[]]
    with sqlite3.connect(os.path.join('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/database/',db_id,'%s.sqlite'%db_id)) as conn:
        conn.text_factory = lambda x:str(x, 'latin1')
        cursor = conn.cursor()
        for (tab_id, col), col_type in zip(db['column_names_original'][1:], db['column_types'][1:]):
            tab = db['table_names_original'][tab_id]
            vals = get_unique_column_values(conn, tab, col, col_type)
            if not vals:
                db_col_values.append([[], [], [], [], []])
                db_col_value_maps.append([])
                continue
            if col_type in ['number', 'time']:
                sub_words = tokenizer('{} {}'.format(vals[0], vals[1]), add_special_tokens=False)['input_ids']
                db_col_values.append([vals, [], [], sub_words, [1]*len(sub_words)])
                db_col_value_maps.append([])
            elif col_type == 'boolean':
                sub_words = [tokenizer.convert_tokens_to_ids('true'), tokenizer.convert_tokens_to_ids('false')]
                db_col_values.append([[0, 1], [], [], sub_words, [1]*len(sub_words)])
                db_col_value_maps.append(sub_words)
            else:
                if col_type == 'others':
                    if vals and (isinstance(vals[0], int) or isinstance(vals[0], float)):
                        sub_words = tokenizer('{} {}'.format(min(vals), max(vals)), add_special_tokens=False)['input_ids']
                        db_col_values.append([[min(vals), max(vals)], [], [], sub_words, [1]*len(sub_words)])
                        db_col_value_maps.append(sub_words)
                        continue
                if vals and (isinstance(vals[0], int) or isinstance(vals[0], float)):
                    db_col_values.append([[], [], [], [], []])
                    db_col_value_maps.append([])
                    print('------error------', tab, col)
                    continue
                nxt_mask, words, cell_values, sub_words_count = prepare_col_values(vals)
                sub_words = list(sub_words_count.keys())
                db_col_values.append([[], list(words), list(cell_values), sub_words, [sub_words_count[sub_word] for sub_word in sub_words]])
                db_col_value_maps.append(sub_words)
                for token_id, mask in nxt_mask.items():
                    if token_id not in db_nxt_mask:
                        db_nxt_mask[token_id] = mask
                    else:
                        db_nxt_mask[token_id] = db_nxt_mask[token_id]|mask
            print(tab,col,len(db_col_values[-1][1]))
    assert len(db_col_values) == len(db['column_names_original'])
    db_nxt_mask[tokenizer.unk_token_id] = [k for k in db_nxt_mask if k!=tokenizer.bos_token_id]+[tokenizer.unk_token_id,tokenizer.eos_token_id]
    db_values[db_id] = [db_col_values, {k:list(v)+[tokenizer.unk_token_id] for k,v in db_nxt_mask.items()}, db_col_value_maps]
    print(len(db_nxt_mask))

 :   0%|          | 0/2000 [00:00<?, ?it/s]

perpetrator
perpetrator Perpetrator_ID 0
perpetrator People_ID 0
perpetrator Date 16
perpetrator Year 0
perpetrator Location 5
perpetrator Country 5
perpetrator Killed 0
perpetrator Injured 0
people People_ID 0
people Name 21
people Height 0
people Weight 0
no such column: Home_Town Home_Town people
70
college_2
classroom building 20
classroom room_number 27
classroom capacity 0
department dept_name 23
department building 16
department budget 0
course course_id 200
course title 212
course dept_name 23
course credits 0
instructor ID 50
instructor name 52
instructor dept_name 20
instructor salary 0
section course_id 85
section sec_id 3
section semester 2
section year 0
section building 18
section room_number 25
section time_slot_id 16
teaches ID 31
teaches course_id 85
teaches sec_id 3
teaches semester 2
teaches year 0


 :   0%|          | 0/2000 [00:00<?, ?it/s]

student ID 2000
student name 1569
student dept_name 23
student tot_cred 0


 :  58%|█████▊    | 1164/2000 [00:00<00:00, 11636.54it/s]

...

building Height_feet 0
building Floors 0
Institution Institution_id 11
Institution Institution 21
Institution Location 24
Institution Founded 0
Institution Type 3
Institution Enrollment 0
Institution Team 12
Institution Primary_Conference 11
Institution building_id 7
protein common_name 12
protein protein_name 16
protein divergence_from_human_lineage 0
protein accession_number 10
protein sequence_length 0
protein sequence_identity_to_human_protein 6
protein Institution_id 4
186
school_player
school School_ID 0
school School 11
school Location 8
school Enrollment 0
school Founded 0
school Denomination 4
school Boys_or_Girls 11
school Day_or_Boarding 3
school Year_Entered_Competition 0
school School_Colors 10
school_details School_ID 0
school_details Nickname 7
school_details Colors 10
school_details League 1
school_details Class 4
school_details Division 4
school_performance School_Id 0
school_performance School_Year 44
school_performance Class_A 22
school_performance Class_AA 21
player

 : 100%|██████████| 15/15 [00:00<00:00, 8807.86it/s]

Cartoon Title 38
Cartoon Directed_by 6
Cartoon Written_by 18
Cartoon Original_air_date 15
Cartoon Production_code 0
Cartoon Channel 9
210
wrestler
wrestler Wrestler_ID 0
wrestler Name 22
wrestler Reign 2
wrestler Days_held 10
wrestler Location 8
wrestler Event 5
Elimination Elimination_ID 6
Elimination Wrestler_ID 6
Elimination Team 3
Elimination Eliminated_By 6
Elimination Elimination_Move 15
Elimination Time 12
112
customer_complaints
Staff staff_id 0
Staff gender 2
Staff first_name 7
Staff last_name 7
Staff email_address 17
Staff phone_number 25
Customers customer_id 0
Customers customer_type_code 6
Customers address_line_1 31
Customers address_line_2 11
Customers town_city 11
Customers state 7
Customers email_address 17
Customers phone_number 28
Products product_id 0
Products parent_product_id 0
Products product_category_code 3
Products date_product_first_available 0
Products date_product_discontinued 0
Products product_name 6
Products product_description 10
Products product_price 


 : 100%|██████████| 2/2 [00:00<00:00, 4576.44it/s]

Contacts last_name 15
Contacts contact_phone 49
Customer_Address_History customer_id 0
Customer_Address_History address_id 0
Customer_Address_History date_from 0
Customer_Address_History date_to 0
Customer_Orders order_id 0
Customer_Orders customer_id 0
Customer_Orders order_date 0
Customer_Orders order_status_code 2
Order_Items order_item_id 0
Order_Items order_id 0
Order_Items product_id 0
Order_Items order_quantity 8
460
company_1
works_on Essn 0
works_on Pno 0
works_on Hours 0
employee Fname 8
employee Minit 8
employee Lname 8
employee Ssn 0
employee Bdate 23
employee Address 23
employee Sex 2
employee Salary 0
employee Super_ssn 0
employee Dno 0
department Dname 3
department Dnumber 0
department Mgr_ssn 0
department Mgr_start_date 9
project Pname 6
project Pnumber 0
project Plocation 4
project Dnum 0
dependent Essn 0
dependent Dependent_name 6
dependent Sex 2
dependent Bdate 17
dependent Relationship 3
dept_locations Dnumber 0
dept_locations Dlocation 4
136
workshop_paper
workshop

In [102]:
# with open('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/tables_column_values.json','w') as f:
#     json.dump(db_values, f)
with open('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/tables_column_values.json','r') as f:
    db_values = json.load(f)

In [107]:
tokenizer.convert_tokens_to_ids("3")

1017

In [108]:
db_values['wta_1'][0]

[[[], [], [], [], []],
 [[200001, 220672],
  [],
  [],
  [2456, 24096, 10545, 2575, 2581, 2475],
  [1, 1, 1, 1, 1, 1]],
 [[],
  ['blair',
   'futaba',
   'gajane',
   'sheya',
   'ruken',
   'farley',
   'emiliana',
   'britt',
   'tamijean',
   'cheli',
   'titia',
   'luh',
   'anne',
   'petrona',
   'silvia',
   'joany',
   'azaliya',
   'tiffini',
   'jette',
   'lili',
   'krishnali',
   'gozal',
   'qing',
   'beatris',
   'bibiane',
   'elisa',
   'frederika',
   'furuza',
   'nimom',
   'tanysha',
   'meiting',
   'sandrine',
   'micheline',
   'ikuko',
   'rasika',
   'josepha',
   'aysu',
   'katrin',
   'martina',
   'saroop',
   'alita',
   'neena',
   'kelesitse',
   'roosh',
   'yuuki',
   'ivy',
   'bojana',
   'myrte',
   'najah',
   'nazari',
   'linda',
   'jaime',
   'advaitas',
   'victoria',
   'nikolette',
   'simran',
   'fumi',
   'oormila',
   'nian',
   'macie',
   'suleika',
   'daleidis',
   'ivone',
   'shadi',
   'janina',
   'yelyzaveta',
   'charmaine',

In [106]:
db_values['wta_1'][1][str(tokenizer.convert_tokens_to_ids("3"))]

[2, 1007, 1011, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 3]

In [95]:
datasets = {'atis':['dev'],
            'geography':['train','dev'],
            'restaurants':[str(x) for x in range(10)],
            'scholar':['train','dev'],
            'imdb':[str(x) for x in range(10)],
            'yelp':[str(x) for x in range(10)],
            'advising':['train','dev'],
            'academic':[str(x) for x in range(10)]}
datadir = '/home/t-xiaden/workspace/text2sql-data/data'

In [97]:
db_values = {}
global_nxt_mask = {
    tokenizer.bos_token_id: set([
        tokenizer.convert_tokens_to_ids('true'),
        tokenizer.convert_tokens_to_ids('false'),
        tokenizer.convert_tokens_to_ids('0'),
        tokenizer.convert_tokens_to_ids('1'),
        tokenizer.convert_tokens_to_ids('2'),
        tokenizer.convert_tokens_to_ids('3'),
    ]),
    tokenizer.convert_tokens_to_ids('true'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('false'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('0'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('1'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('2'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('3'): set([tokenizer.eos_token_id]),
}
for db_id in datasets:
    print(db_id)
    db_col_values = [[[], [], [], [], []]]
    db_nxt_mask = copy.deepcopy(global_nxt_mask)
    db_col_value_maps = [[]]
    with open('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/%s_tables.json'%db_id,'r') as f:
        db = json.load(f)
        assert len(db) == 1
        db = db[0]
    with sqlite3.connect('/home/t-xiaden/workspace/text2sql-data/data/%s.db'%db_id) as conn:        
        conn.text_factory = lambda x:str(x, 'latin1')
        cursor = conn.cursor()
        for (tab_id, col), col_type in zip(db['column_names_original'][1:], db['column_types'][1:]):
            tab = db['table_names_original'][tab_id]
            vals = get_unique_column_values(conn, tab, col, col_type)
            if not vals:
                db_col_values.append([[], [], [], [], []])
                db_col_value_maps.append([])
                continue
            if col_type in ['number', 'time']:
                sub_words = tokenizer('{} {}'.format(vals[0], vals[1]), add_special_tokens=False)['input_ids']
                db_col_values.append([vals, [], [], sub_words, [1]*len(sub_words)])
                db_col_value_maps.append([])
            elif col_type == 'boolean':
                sub_words = [tokenizer.convert_tokens_to_ids('true'), tokenizer.convert_tokens_to_ids('false')]
                db_col_values.append([[0, 1], [], [], sub_words, [1]*len(sub_words)])
                db_col_value_maps.append(sub_words)
            else:
                if col_type == 'others':
                    if vals and (isinstance(vals[0], int) or isinstance(vals[0], float)):
                        sub_words = tokenizer('{} {}'.format(min(vals), max(vals)), add_special_tokens=False)['input_ids']
                        db_col_values.append([[min(vals), max(vals)], [], [], sub_words, [1]*len(sub_words)])
                        db_col_value_maps.append(sub_words)
                        continue
                if vals and (isinstance(vals[0], int) or isinstance(vals[0], float)):
                    db_col_values.append([[], [], [], [], []])
                    db_col_value_maps.append([])
                    print('------error------', tab, col)
                    continue
                nxt_mask, words, cell_values, sub_words_count = prepare_col_values(vals)
                sub_words = list(sub_words_count.keys())
                db_col_values.append([[], list(words), list(cell_values), sub_words, [sub_words_count[sub_word] for sub_word in sub_words]])
                db_col_value_maps.append(sub_words)
                for token_id, mask in nxt_mask.items():
                    if token_id not in db_nxt_mask:
                        db_nxt_mask[token_id] = mask
                    else:
                        db_nxt_mask[token_id] = db_nxt_mask[token_id]|mask
            print(tab,col,len(db_col_values[-1][1]))
    assert len(db_col_values) == len(db['column_names_original'])
    db_nxt_mask[tokenizer.unk_token_id] = [k for k in db_nxt_mask if k!=tokenizer.bos_token_id]+[tokenizer.unk_token_id,tokenizer.eos_token_id]
    db_values[db_id] = [db_col_values, {k:list(v)+[tokenizer.unk_token_id] for k,v in db_nxt_mask.items()}, db_col_value_maps]
    print(len(db_nxt_mask))

 :   0%|          | 0/504 [00:00<?, ?it/s]

atis
aircraft aircraft_code 43
aircraft aircraft_description 85
aircraft manufacturer 19
aircraft basic_type 30
aircraft engines 0
aircraft propulsion 2
aircraft wide_body 2
aircraft wing_span 0
aircraft length 0
aircraft weight 0
aircraft capacity 0
aircraft pay_load 0
aircraft cruising_speed 0
aircraft range_miles 0
aircraft pressurized 2
airline airline_code 46
airline airline_name 63
airline note 7
airport airport_code 52
airport airport_name 92
airport airport_location 89
airport state_code 26
airport country_name 2
airport time_zone_code 4
airport minimum_connect_time 0
airport_service city_code 46
airport_service airport_code 52
airport_service miles_distant 0
airport_service direction 8
airport_service minutes_distant 0
city city_code 46
city city_name 57
city state_code 26
city country_name 2
city time_zone_code 4
class_of_service booking_class 19
class_of_service rank 0
class_of_service class_description 20
code_description code 7
code_description description 26
compartment_c

 :   0%|          | 0/18708 [00:00<?, ?it/s]

equipment_sequence aircraft_code_sequence 44
equipment_sequence aircraft_code 43
fare fare_id 0
fare from_airport 52
fare to_airport 52
fare fare_basis_code 60
fare fare_airline 28
fare restriction_code 17
fare one_direction_cost 0
fare round_trip_cost 0
fare round_trip_required 2
fare_basis fare_basis_code 60
fare_basis booking_class 19
fare_basis class_type 6
fare_basis premium 2
fare_basis economy 2
fare_basis discounted 2
fare_basis night 2
fare_basis season 4
fare_basis basis_days 4
flight flight_id 0
flight flight_days 9
flight from_airport 52
flight to_airport 52
flight departure_time 0
flight arrival_time 0


 : 100%|██████████| 4/4 [00:00<00:00, 4904.18it/s]

...

publication title 391023


 : 100%|██████████| 2343734/2343734 [1:42:06<00:00, 382.58it/s]


publication abstract 1082772
publication year 0
publication cid 0
publication jid 0
publication reference_num 0
publication citation_num 0


 : 100%|██████████| 2494136/2494136 [07:09<00:00, 5813.80it/s]


publication doi 1324380
publication_keyword pid 0
publication_keyword kid 0
writes aid 0
writes pid 0
27396


In [98]:
db_values.keys()

dict_keys(['atis', 'geography', 'restaurants', 'scholar', 'imdb', 'yelp', 'advising', 'academic'])

In [99]:
for db_id in db_values:
    with open('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/%s_tables_column_values.json'%db_id,'w') as f:
        json.dump({db_id:db_values[db_id]}, f)

In [100]:
treqs_db_values = {}

global_nxt_mask = {
    tokenizer.bos_token_id: set([
        tokenizer.convert_tokens_to_ids('true'),
        tokenizer.convert_tokens_to_ids('false'),
        tokenizer.convert_tokens_to_ids('0'),
        tokenizer.convert_tokens_to_ids('1'),
        tokenizer.convert_tokens_to_ids('2'),
        tokenizer.convert_tokens_to_ids('3'),
    ]),
    tokenizer.convert_tokens_to_ids('true'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('false'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('0'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('1'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('2'): set([tokenizer.eos_token_id]),
    tokenizer.convert_tokens_to_ids('3'): set([tokenizer.eos_token_id]),
}
for db_id in ['treqs']:
    print(db_id)
    db_col_values = [[[], [], [], [], []]]
    db_nxt_mask = copy.deepcopy(global_nxt_mask)
    db_col_value_maps = [[]]
    with open('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/%s_tables.json'%db_id,'r') as f:
        db = json.load(f)
        assert len(db) == 1
        db = db[0]
    with sqlite3.connect('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/database/{}/{}.sqlite'.format(db_id,db_id)) as conn:
        conn.text_factory = lambda x:str(x, 'latin1')
        cursor = conn.cursor()
        for (tab_id, col), col_type in zip(db['column_names_original'][1:], db['column_types'][1:]):
            tab = db['table_names_original'][tab_id]
            vals = get_unique_column_values(conn, tab, col, col_type)
            if not vals:
                db_col_values.append([[], [], [], [], []])
                db_col_value_maps.append([])
                continue
            if col_type in ['number', 'time']:
                sub_words = tokenizer('{} {}'.format(vals[0], vals[1]), add_special_tokens=False)['input_ids']
                db_col_values.append([vals, [], [], sub_words, [1]*len(sub_words)])
                db_col_value_maps.append([])
            elif col_type == 'boolean':
                sub_words = [tokenizer.convert_tokens_to_ids('true'), tokenizer.convert_tokens_to_ids('false')]
                db_col_values.append([[0, 1], [], [], sub_words, [1]*len(sub_words)])
                db_col_value_maps.append(sub_words)
            else:
                if col_type == 'others':
                    if vals and (isinstance(vals[0], int) or isinstance(vals[0], float)):
                        sub_words = tokenizer('{} {}'.format(min(vals), max(vals)), add_special_tokens=False)['input_ids']
                        db_col_values.append([[min(vals), max(vals)], [], [], sub_words, [1]*len(sub_words)])
                        db_col_value_maps.append(sub_words)
                        continue
                if vals and (isinstance(vals[0], int) or isinstance(vals[0], float)):
                    db_col_values.append([[], [], [], [], []])
                    db_col_value_maps.append([])
                    print('------error------', tab, col)
                    continue
                nxt_mask, words, cell_values, sub_words_count = prepare_col_values(vals)
                sub_words = list(sub_words_count.keys())
                db_col_values.append([[], list(words), list(cell_values), sub_words, [sub_words_count[sub_word] for sub_word in sub_words]])
                db_col_value_maps.append(sub_words)
                for token_id, mask in nxt_mask.items():
                    if token_id not in db_nxt_mask:
                        db_nxt_mask[token_id] = mask
                    else:
                        db_nxt_mask[token_id] = db_nxt_mask[token_id]|mask
            print(tab,col,len(db_col_values[-1][1]))
    assert len(db_col_values) == len(db['column_names_original'])
    db_nxt_mask[tokenizer.unk_token_id] = [k for k in db_nxt_mask if k!=tokenizer.bos_token_id]+[tokenizer.unk_token_id,tokenizer.eos_token_id]
    treqs_db_values[db_id] = [db_col_values, {k:list(v)+[tokenizer.unk_token_id] for k,v in db_nxt_mask.items()}, db_col_value_maps]
    print(len(db_nxt_mask))
    
for db_id in treqs_db_values:
    with open('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/%s_tables_column_values.json'%db_id,'w') as f:
        json.dump({db_id:treqs_db_values[db_id]}, f)

 :   0%|          | 0/508 [00:00<?, ?it/s]

treqs
DEMOGRAPHIC SUBJECT_ID 0
DEMOGRAPHIC HADM_ID 0
DEMOGRAPHIC NAME 185
DEMOGRAPHIC MARITAL_STATUS 4
DEMOGRAPHIC AGE 0
DEMOGRAPHIC DOB 107
DEMOGRAPHIC GENDER 2
DEMOGRAPHIC LANGUAGE 9
DEMOGRAPHIC RELIGION 21
DEMOGRAPHIC ADMISSION_TYPE 4
DEMOGRAPHIC DAYS_STAY 0
DEMOGRAPHIC INSURANCE 6
DEMOGRAPHIC ETHNICITY 23
DEMOGRAPHIC EXPIRE_FLAG 0
DEMOGRAPHIC ADMISSION_LOCATION 22
DEMOGRAPHIC DISCHARGE_LOCATION 28
DEMOGRAPHIC DIAGNOSIS 156
DEMOGRAPHIC DOD 50
DEMOGRAPHIC DOB_YEAR 0
DEMOGRAPHIC DOD_YEAR 0
DEMOGRAPHIC ADMITTIME 117
DEMOGRAPHIC DISCHTIME 109
DEMOGRAPHIC ADMITYEAR 0
DIAGNOSES SUBJECT_ID 0
DIAGNOSES HADM_ID 0


 :  87%|████████▋ | 441/508 [00:00<00:00, 4406.76it/s]

DIAGNOSES ICD9_CODE 508
DIAGNOSES SHORT_TITLE 910


 :   0%|          | 0/707 [00:00<?, ?it/s]

...

PRESCRIPTIONS FORMULARY_DRUG_CD 741
PRESCRIPTIONS ROUTE 30
PRESCRIPTIONS DRUG_DOSE 285
LAB SUBJECT_ID 0
LAB HADM_ID 0
LAB ITEMID 0


 :  22%|██▏       | 989/4577 [00:00<00:00, 9887.84it/s]

LAB CHARTTIME 125
LAB FLAG 2


 : 100%|██████████| 3/3 [00:00<00:00, 5216.80it/s]


LAB VALUE_UNIT 2455
LAB LABEL 212
LAB FLUID 12
LAB CATEGORY 4
3194


In [ ]:
'texas' in db_values['geography'][0][1][1]

In [61]:
[x for x in db_values['scholar'][0][2][1] if re.search(r'\bmonte carlo\b',x)]

['monte carlo path',
 'quantum monte carlo',
 'monte carlo renormalization',
 'monte carlo smart',
 'h j pradlwarter monte carlo',
 'monte carlo rough',
 'monte carlo - free',
 'monte carlo chain',
 'monte carlo studies',
 'monte carlo algorithms',
 'monte carlo on gpus',
 'monte carlo study',
 'monte carlo method',
 'a monte carlo',
 'monte carlo calculations',
 'monte carlo methods',
 'monte carlo - based',
 'monte carlo sampling',
 'monte carlo hamiltonian',
 'monte carlo',
 'monte carlo monte',
 'd w pipes monte carlo',
 'monte carlo bayesian',
 'monte carlo integration',
 'monte carlo implemen',
 'monte carlo simulation',
 'sequential monte carlo',
 'monte carlo workshop',
 'monte carlo simulations',
 'mcml - - monte carlo',
 'monte carlo ray',
 'monte carlo algorithm',
 'monte carlo stratego',
 'monte carlo schach',
 'f pillichshammer monte carlo',
 'markovian monte carlo',
 'monte carlo hotel',
 'monte carlo evidences',
 'monte carlo sim',
 'monte carlo resort',
 'monte carlo si

In [43]:
'born' in db_values['imdb'][0][3][1]

True

In [64]:
db_id = 'scholar'
with sqlite3.connect(os.path.join('/home/t-xiaden/workspace/featurestorage/data/spider-20190205/database/',db_id,'%s.sqlite'%db_id)) as conn:
    conn.text_factory = lambda x:str(x, 'latin1')
    cursor = conn.cursor()
    print(cursor.execute('select * from author where authorName=\'monte carlo simulations\' COLLATE NOCASE').fetchall())

[(4709499, 'Monte Carlo Simulations')]
